In [1]:
from google.colab import drive
import pandas as pd
import numpy
drive.mount('/content/drive')
songs = pd.read_csv('/content/drive/My Drive/Colab Notebooks/yataros1.csv',header=None)

dataset_sec_rec=songs.copy()
dataset_sec_rec=dataset_sec_rec.transpose()

dataset_sec_rec.drop(0, axis=1, inplace = True)
dataset_sec_rec = dataset_sec_rec.iloc[1:]
dataset_sec_rec = dataset_sec_rec.reset_index(drop=True)
dataset_sec_rec = dataset_sec_rec.to_numpy()
dataset_sec_rec=dataset_sec_rec.astype(numpy.float64)

# songs=songs.transpose()
# print(songs.isnull().sum())

songs.drop(0, axis=1, inplace = True)
songs = songs.iloc[1:]
songs = songs.reset_index(drop=True)
dataset = songs.to_numpy()

# dataset=dataset[0:10,0:12]
dataset=dataset.astype(numpy.float64)
n=numpy.size(dataset,axis=0)
# print(numpy.size(dataset,axis=0),numpy.size(dataset,axis=1))
# print(dataset)
cents_graph_global=[]
# print(n)

Mounted at /content/drive


In [2]:
#CPU PARALELO
import numpy as np
from timeit import default_timer
import pandas as pd
import multiprocessing
import time

def encontrar_minimo_por_columna(submatriz):
    minimo,I = np.min(submatriz),np.argmin(submatriz)
    return minimo,I

def find_m_h(chunks):
    JJ,d,i,n=chunks
    if(len(JJ)==0):
        return i
    else:
        M_h=np.full((1,n),np.inf)
        maxs=np.max(d,axis=1)
        for j in range(len(maxs)):
            jj=JJ[j]
            M_h[0][jj]=maxs[j]

        return M_h

def encontrar_minimo_en_M_h(submatriz):

    I = np.argmin(submatriz)

    return I


def kcentres(d,k=1,nrep=1):

    elapsed_time=0
    m,m2=d.shape
    dopt=np.inf
    cents_graph_cpu_parall=[]
    for tri in range(1,nrep+1):
        M_h=np.full((k,m),np.inf)
        # K=np.random.permutation(m)
        # K=K[0:k]
        K=np.array([3, 14, 35, 24, 39 ], dtype=np.int64)
        J=np.array([-1]*k)

        start_time = time.time()
        while True:
            # Crear un pool de procesos y calcular distancias en paralelo
            pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())

            resultados = pool.map(encontrar_minimo_por_columna, [(d[K][:,i]) for i in range(m)])
            dm,I=zip(*resultados)
            I= [np.where(K[i]==K) for i in I]
            I=np.array(I)

            # print(I.flatten())

            # print("dm: ",dm)

            input_values_for_mh = [(np.where(I==i)[0],d[np.where(I==i)[0]][:,np.where(I==i)[0]],i,m) for i in range(k)]

            stack = pool.map(find_m_h, input_values_for_mh)  # Calcular distancias en paralelo

            M_h = np.vstack(stack)

            J=pool.map(encontrar_minimo_en_M_h, [(M_h[i,:]) for i in range(k)])
            J=np.array(J)
            pool.close()  # Cerrar el pool de procesos
            pool.join()   # Esperar a que todos los procesos terminen

            k=len(J)
            if(k==0):
                print('kcentres,es fails as some objects are identical: add some noise')
                return
            if(np.all(K==J)):
                #K centres - convergen
                dmin=np.max(dm)
                labs=I
                break
            K=J

        finish_time = time.time()

        elapsed_time+=(finish_time-start_time)

        #almacena los centros más optimos en jopt.
        if(dmin<=dopt):
            dopt=dmin
            labels=np.transpose(labs)
            jopt=J

    #Determina los mejores centros sobre los N intentos.
    dm=np.zeros(k)
    # print(labels.flatten())
    for i in range(k):
        # L=np.where(labels==i)[0]
        L=np.where(labels==i)
        dm[i]=np.max(d[jopt[i],L])

    print("Tiempo de K-centres: ",elapsed_time/nrep)
    return labels,jopt,dm

def calculate_distances_parallel(chunk):
    # Esta función calcula las distancias entre un subconjunto de objetos
    dataset,start,end = chunk
    num_objects=dataset.shape[0]
    distances_chunk = np.zeros((end - start, num_objects))
    for i in range(start, end):
        for j in range(num_objects):
            distances_chunk[i - start, j] = np.linalg.norm(dataset[i] - dataset[j])
    return distances_chunk

if __name__=="__main__":

    #songs = dataset_sec_rec

    num_objects=dataset_sec_rec.shape[0]
    num_processes = multiprocessing.cpu_count()  # Obtener el número de núcleos de la CPU
    chunk_size = num_objects // num_processes  # Dividir los objetos en lotes

    start_time = time.time()
    chunks = [(dataset_sec_rec,i, min(i + chunk_size, num_objects)) for i in range(0, num_objects, chunk_size)]

    # Crear un pool de procesos y calcular distancias en paralelo
    pool = multiprocessing.Pool(processes=num_processes)
    distances_chunks = pool.map(calculate_distances_parallel, chunks)  # Calcular distancias en paralelo
    pool.close()  # Cerrar el pool de procesos
    pool.join()   # Esperar a que todos los procesos terminen

    # Combinar los resultados de los procesos en una matriz de distancias
    disi_mat = np.vstack(distances_chunks)


    finish_time = time.time()

    # # print(disi_mat)
    labels,jopt,dm=kcentres(disi_mat,5,25)

    # print("K.CENTRES-paralelo")
    print("Tiempo de Matriz de distancias:  ",finish_time-start_time)
    print("\n",labels,dm)

    print("\nCENTROIDES: ",jopt)

Tiempo de K-centres:  0.5530390739440918
Tiempo de Matriz de distancias:   0.13288474082946777

 [[[2 3 3 3 3 1 4 1 1 4 1 1 1 1 1 1 1 2 0 0 0 2 0 1 1 1 1 1 1 1 1 4 1 3 1
   4 1 1 4 1]]] [1.87170367e+08 1.00843105e+05 3.03671419e+05 9.23161558e+04
 1.00696773e+05]

CENTROIDES:  [18 24  0  2  9]
